In [1]:
from common.data_pipeline.dataset import DatasetCompiler
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm

datasets = DatasetCompiler(datasets=[mmcbnu(), fvusm()])
datasets = datasets.compile_datasets()

Concatenating train set
(8332, 10, 10)
(8332, 1, 1)
Concatenating test set
(2381, 10, 10)
(2381, 1, 1)
Concatenating validation set
(1191, 10, 10)
(1191, 1, 1)
